In [9]:
import frost_sta_client as fsc
import arrow
import pyecharts
import pandas as pd

In [2]:
url = "http://94.154.11.74/frost/v1.1/"
service = fsc.SensorThingsService(url)
MDT = service.multi_datastreams().find('RudnMeteoRawDataMultiStream')


In [ ]:
# function get observations entity list and converts result and date to pandas dataframe
def FrostObservationsToPandas(Observations): 
    

In [88]:
Observations = MDT.get_observations().query().filter("phenomenonTime ge 2024-10-01T00:00:00+00:00 and phenomenonTime lt 2024-10-12T00:00:00+00:00").list()

result_list=[]
for observation in Observations:
    result_list.append(observation.result+[arrow.get(observation.phenomenon_time).format('YYYY-MM-DD HH:mm:ss')])



In [89]:
Observations = MDT.get_observations().query().filter("phenomenonTime ge 2024-10-01T00:00:00+00:00 and phenomenonTime lt 2024-10-12T00:00:00+00:00").list()

result_list = []
for observation in Observations:
    result_list.append(observation.result+[arrow.get(observation.phenomenon_time).to('Europe/Moscow').format('YYYY-MM-DD HH:mm:ss')])
df = pd.DataFrame(result_list, columns = ['WinDir_min', 'WinDir_mean','WinDir_max',
                                          'WinSpeed_min','WinSpeed_mean','WinSpeed_max',
                                          'Tair','RH','P_atm','Pcum','Timestamp']) 
print(result_list)
print(df)
#Dn – минимальное значение направления ветра
#Dm – среднее значение направления ветра
#Dx – максимальное значение направления ветра
#Sn – минимальное значение скорости ветра
#Sm – среднее значение скорости ветра
#Sx – максимальное значение скорости ветра
#Ta – температура воздуха (0С)
#Ua – влажность воздуха (%)
#Pa – атмосферное давление (гПа
#Rc – количество осадков (мм)

[[32, 63, 75, 1.3, 1.6, 1.7, 10.5, 85, 988.4, 27.7, '2024-10-05 21:27:19'], [24, 65, 59, 0.8, 1.5, 2.1, 10.5, 84.8, 988.4, 27.7, '2024-10-05 21:28:19'], [30, 71, 90, 0.6, 1.5, 1.4, 10.5, 84.8, 988.5, 27.7, '2024-10-05 21:29:19'], [65, 53, 90, 1, 0.9, 1.8, 10.5, 84.9, 988.5, 27.7, '2024-10-05 21:30:19'], [31, 49, 46, 2, 1.4, 2.5, 10.4, 85.1, 988.4, 27.7, '2024-10-05 21:31:19'], [83, 62, 97, 1.3, 1.8, 2.3, 10.4, 85, 988.4, 27.7, '2024-10-05 21:32:19'], [46, 57, 90, 2, 2, 2.7, 10.5, 84.8, 988.4, 27.7, '2024-10-05 21:33:19'], [56, 51, 102, 0.5, 1.5, 1.2, 10.5, 85, 988.3, 27.7, '2024-10-05 21:34:19'], [0, 57, 65, 0, 1.3, 0.7, 10.5, 85.2, 988.3, 27.7, '2024-10-05 21:35:19'], [67, 62, 78, 2, 1.6, 2.8, 10.4, 85.4, 988.3, 27.7, '2024-10-05 21:36:19'], [46, 77, 69, 1.4, 1.2, 1.7, 10.5, 85.2, 988.4, 27.7, '2024-10-05 21:37:19'], [15, 70, 86, 0.5, 1.4, 1.2, 10.5, 85.3, 988.3, 27.7, '2024-10-05 21:38:19'], [50, 64, 63, 1, 1.2, 1.4, 10.5, 85.4, 988.4, 27.7, '2024-10-05 21:39:19'], [48, 55, 61, 0.8, 

In [90]:
import pyecharts.options as opts
from pyecharts.charts import Line, Grid
all_data = result_list
(
    Line()
    .add_xaxis(xaxis_data=[item[10] for item in all_data])
    .add_yaxis(
        series_name="Температура воздуха, С",
        y_axis=[item[6] for item in all_data],
        yaxis_index=0,
        is_smooth=False,
        is_symbol_show=False,
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="Метеостанция АТИ РУДН"),
        tooltip_opts=opts.TooltipOpts(trigger="axis"),
        datazoom_opts=[
            opts.DataZoomOpts(xaxis_index=0, yaxis_index=1),
            opts.DataZoomOpts(type_="slider"),
            opts.DataZoomOpts(orient="vertical", pos_left=20)
        ],
        visualmap_opts=opts.VisualMapOpts(
            pos_top="10",
            pos_right="10",
            is_piecewise=True,
            pieces=[
                {"gt": 0, "lte": 5, "color": "rgb(19, 124, 209)"},
                {"gt": 5, "lte": 10, "color": "#08d0e7"},
                {"gt": 10, "lte": 15, "color": "#0ef6df"},
                {"gt": 15, "lte": 20, "color": "#19c485"},
                {"gt": 20, "lte": 30, "color": "#249138"},
                {"gt": 30, "color": "#d97b1c"},
            ],
            out_of_range={"color": "#999"},
        ),
        xaxis_opts=opts.AxisOpts(type_="category"),
        yaxis_opts=opts.AxisOpts(
            type_="value",
            name_location="start",
            is_scale=True,
            axistick_opts=opts.AxisTickOpts(is_inside=False),
        ),
    )
    .set_series_opts(
        markline_opts=opts.MarkLineOpts(
            data=[
                {"yAxis": 5},
                {"yAxis": 10},
                {"yAxis": 15},
                {"yAxis": 20},
                {"yAxis": 30},
            ],symbol='none',
            label_opts=opts.LabelOpts(position="end"),
        )
    )
    .render_notebook()
)